In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **DOING ANALYSIS ON THE MAIN FILE**

In [29]:
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns

In [84]:
train_df=pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')
train_df.head(20)

In [85]:
print("number of rows and colums")
print(train_df.shape[0]," ",train_df.shape[1])

## **1) Finding the number of columns and its names that have more than 70% missing values, 10 to 70% and less than 10% missing values**

In [89]:

lst=train_df.isnull().sum()
cnt=0
cnt1=0
cnt2=0
l3=[]
for i in range(len(lst)):
    if lst[i] <30000:
        cnt+=1
    if lst[i] >30000 and lst[i]<200000:
        cnt1+=1
    if lst[i] >200000:
        l3.append(i)
        cnt2+=1
print(cnt," ",cnt1," ",cnt2)
#train_df.drop(train_df.columns[delst],axis=1,inplace=True)

17 columns have more than 70% data missing,
40 columns have 10% data missing and 65 columns have missing values less than 10%



## **2) Filling missing values**

### We drop columns with more than 70% issing values, impute values for quantitative variables using **ridge regression** and imputed using mode for columns with categorical (both string and numeric) variables

**a) dropping columns with >70% missing values**

In [90]:
#drop columns with >70% missing values
train_df.drop(train_df.columns[l3],axis=1,inplace=True)


**b) filling categorical variables with mode**

In [34]:
#for categorical values, fill with mode
cat_vals=train_df.select_dtypes('object').nunique().index.tolist()
train_df.select_dtypes('object').nunique()
for i in cat_vals:
    train_df[i].fillna(train_df[i].mode()[0], inplace=True)

Now that columns with many missing values are dropped and columns with non numeric data have been imputed with mode, missing values in numeric data need to be taken care of

In [91]:
#find all numeric data
num_cols = train_df.select_dtypes(include=np.number).columns.tolist()
print(len(num_cols))
#there are 90 columns with numeric data
df_num= train_df.select_dtypes(include=np.number) #make a seperate dataset with them


In [36]:
df_num = df_num.iloc[: , 1:] # remove the first column with id as it will not help in imputing values

**A summary of all the numerical values**

In [37]:
df_num.describe()

**finding the columns which have quantitative data and qualititive data** 

In [38]:
#divide the numerical columns into numeric categorical and continous variable columns. If the column has less than 10 distinct values, it is treated to be categorical data
num_cols.pop(0)
cont_cols=[]
dis_cols=[]
for i in num_cols:
    if(len(df_num[i].unique())>10):
        cont_cols.append(i)
    else:
        dis_cols.append(i)
        
    

There are 39 columns with numeric classes and 50 columns with qualititive data

Data is heavily skewed towards value 0 for all qualitative variables. Hence we would need to balance the dataset before proceeding to build the model. But for the sake of PCA and continuing with EDA, these variables will be filled using mode.

**c) Filling numeric qualitative  data with mode**

In [40]:
for i in dis_cols:
    train_df[i].fillna(value=train_df[i].mode(),inplace=True)

    

**d) Imputing the remaining variables using ridge regression**

In [41]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [43]:
df_num2= train_df.select_dtypes(include=np.number)
imputer = IterativeImputer(imputation_order='ascending') #by default the estimator used is bayesian ridge regressor
imputer.fit(df_num2)
# transform the dataset
Xtrans = imputer.transform(df_num2)

In [44]:
#once the imputing is done, making a new dataframe of numerical columns 
num_cols = train_df.select_dtypes(include=np.number).columns.tolist()
df_new = pd.DataFrame(data = Xtrans,  
                  columns = num_cols)
df_new.head(10)

## 3) Visualisation

**a) distribution of target variable**

In [47]:
train_df["TARGET"].value_counts()
sns.countplot(train_df['TARGET'])
# 0 means loan repayed, 1 means unpaid
#data is highly unbalanced

**b) distribution of values between the 2 target classes for categorical non numeric data**

In [48]:
#see comparision of defaulters vs non based on different categorical attributes
def show_category_by_target(df, columns):
    for column in columns:
        chart = sns.catplot(x=column, col='TARGET', data=df, kind='count')
        chart.set_xticklabels(rotation=65)

show_category_by_target(train_df, cat_vals)

since the propotion of defaulters in the dataset is less, we cannot conclude anything definite, but it can be noticed that there is same trend/distribution of defaulters and non defaulters in all attributes.

### For numeric data


**c) distribution of age**

In [49]:
#find age of the ppl in dataset
plt.hist(train_df['DAYS_BIRTH']/-365, edgecolor='k',bins=25)
plt.title('Age of Client');
plt.xlabel('Age (years)');
plt.ylabel('Count');
#shows there is almost normal distribution of all age groups

**d) A plot to see the relation between age and defaulters**

In [50]:
age_data=train_df[['TARGET','DAYS_BIRTH']]
age_data['YEARS_BIRTH']=age_data['DAYS_BIRTH']/-365
age_data['bin'] = pd.cut(age_data['YEARS_BIRTH'], 11, include_lowest = True)
age_groups  = age_data.groupby('bin').mean()
plt.bar(age_groups.index.astype(str), 100*age_groups['TARGET'])

# Plot labeling
plt.xticks(rotation=75);
plt.xlabel('Age Group (years)');
plt.ylabel('Failur to Reapy(%)')
plt.title('Failure to Repay by Age Group');
#younger generation most likely to default and not replay loan

**e) denstity plot of all quantitative variables for both the target classes to see if the distribution of values is same for both defaulters and non defaulters**

In [94]:
#density plots of all continous numerical variables


for column in cont_cols:
    print(column)
    sns.kdeplot(df_new.loc[(df_new['TARGET'] == 0.0), column] , label = 'Will Repay')
    sns.kdeplot(df_new.loc[(df_new['TARGET'] == 1.0), column] , label = 'Will Default')
    plt.legend()
    plt.show()

From the above density plots it can be inferred that attributes like EXT_SOURCE_2 and DAYS_SINCE_BIRTH can be used to distinguish between defaulters and non defaulters as the distribution of values in these columns do not overlap. 

**f) Box plot to detect outliers**

In [52]:
#box plot of all numeric data to spot outliers
for column in df_num.columns:
    print(column)
    sns.boxplot(x=df_num[column])
    plt.show()

In [53]:
sns.boxplot(x=df_num['HOUR_APPR_PROCESS_START'])
plt.show()
sns.boxplot(x=df_num['CNT_FAM_MEMBERS'])
plt.show()

## 3)PCA AND DATA REDUCTION

In [70]:
df_new = df_new.iloc[: , 1:]

In [71]:
df_new.shape #original shape

Dividing the dataset into target and feature variables

In [72]:
target_variable = df_new["TARGET"]
features_variable=df_new.drop("TARGET",axis=1)


Since the range of all the variables is different, we standardise them

In [73]:
from sklearn.preprocessing import StandardScaler
standarized_data = StandardScaler().fit_transform(features_variable)


We want the total variance to be 90%

In [75]:
from sklearn import decomposition
pca = decomposition.PCA(n_components = 0.90)

In [76]:
pca_data = pca.fit_transform(standarized_data) 

In [82]:
pca_data.shape
#thus the total shape has reduced from 89 to 47 columns

In [78]:
pca_df = pd.DataFrame(data = pca_data)
pca_df

In [66]:
cat_cols = train_df.select_dtypes('object')

In [80]:
final_df=pd.concat([train_df['SK_ID_CURR'],cat_cols, pca_df], axis=1, join='inner')
final_df.shape


In [81]:
final_df.head(10)